In [1]:
import cv2
import mediapipe as mp
from mediapipe.python._framework_bindings import image_frame
import pandas as pd
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/bin/ffmpeg"
import time
from moviepy.editor import *
import matplotlib.pyplot as plt
import subprocess
import glob, os
import time
initial_dir = os.getcwd()

df = pd.DataFrame()
frames = []

In [2]:
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode


options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='hand_landmarker.task'),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=2,
)
hands = HandLandmarker.create_from_options(options)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
# Convert normalized coordinates to Matplotlib-compatible coordinates
def convert_to_matplotlib_coords(coord, image_height):
    x, y = coord
    return x, image_height - y

def normalize_coordinates(coordinates, target):
    max_x = max(coord[0] for coord in coordinates)
    max_y = max(coord[1] for coord in coordinates)
    
    normalized_coordinates = []
    for x, y in coordinates:
        normalized_x = (x / max_x)
        normalized_y = (y / max_y)
        normalized_coordinates.append([normalized_x, normalized_y])
    
    # Convert coordinates for plotting
    # matplotlib_coords = [convert_to_matplotlib_coords(coord, image_height) for coord in normalized_coordinates]
    # if (target == "v" or target=="y"):
    #     visualize_data(normalized_coordinates, target)
    return normalized_coordinates

def visualize_data(normalized_coordinates, target):
    # Unzip normalized coordinates for plotting
    normalized_x, normalized_y = zip(*normalized_coordinates)
    
    # Plot the normalized coordinates
    plt.figure(figsize=(8, 6))
    plt.scatter(normalized_x, normalized_y, color='blue', label='Normalized Coordinates')
    plt.xlabel('Normalized X')
    plt.xlabel('Normalized X')
    plt.ylabel('Normalized Y')
    plt.title('Normalized Coordinates Plot')
    plt.legend(target)
    plt.grid(True)
    plt.show()

In [23]:
def extract_video(video, target, sequence_id, real_path):
    global frames
    added_rows = 0
    # For webcam input:
    cap = cv2.VideoCapture(video)
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))


    # Calculate the current duration of the video
    video_duration = frame_count / fps
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        

        name = f'{initial_dir}/test_frame.png'
        cv2.imwrite(name, frame)
        mp_image = mp.Image.create_from_file(name)
        hand_landmarker_result = hands.detect(mp_image)

         
        if len(hand_landmarker_result.handedness) > 0:
            row_data = {
                "sequence_id": sequence_id,
                "target": target,
                "file": real_path
            }
            hand_sides = ["Left", "Right"]
            for idx, landmarks in enumerate(hand_landmarker_result.hand_landmarks):
                detected_pixels = []
                hand_side = hand_sides[idx]
                # Iterate through detected hand landmarks
                for landmark_idx, landmark in enumerate(landmarks):
                    x, y = landmark.x, landmark.y
                    detected_pixels.append([x, y])
                    # Draw circles on the frame
                    cv2.circle(frame, (int(x * frame.shape[1]), int(y * frame.shape[0])), 5, (0, 255, 0), -1)

                    
                detected_pixels = normalize_coordinates(detected_pixels, target)
                for i in range(len(detected_pixels)):
                    x, y = detected_pixels[i]
                    row_data[f'x_{hand_side}_hand_{i}'] =  x
                    row_data[f'y_{hand_side}_hand_{i}'] =  y
                
            if (len(hand_landmarker_result.handedness) == 1):
                for i in range(21):
                    x, y = [0, 0]
                    row_data[f'x_{hand_sides[1]}_hand_{i}'] =  x
                    row_data[f'y_{hand_sides[1]}_hand_{i}'] =  y
                    
            added_rows += 1
            frames.append(row_data)
        cv2.imshow('Hand Tracking', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        cv2.waitKey(1)
        if (added_rows == 30):
            break
    if (added_rows == 0):
        print("!! No hand detected in ", real_path)

    cap.release()
    cv2.destroyAllWindows()

In [5]:
def get_length(filename):
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

In [6]:
def change_to_30_fps(video_input_path, video_output_path):
    duration = 1/get_length(video_input_path)

    c = f'ffmpeg -loglevel 0 -y -itsscale {duration} -i "' + video_input_path + f'" -filter:v fps=fps=30 "' + video_output_path + '"'
    subprocess.call(c, shell=True)

In [7]:
sequence_id = 0
validation_sequence_ids = []

def extract_coordinates_from_dir(dir, is_val=False):
    global sequence_id, initial_dir, validation_sequence_ids
    os.chdir(dir)

    output_fps_path = f'{dir}/adjusted_fps_video.mp4'

    for file in glob.glob("*.mp4"):
        if (file == "adjusted_fps_video.mp4"):
            return
            
        input_path = f'{dir}/{file}'
        sequence_id += 1
        if (is_val):
            validation_sequence_ids.append(sequence_id)

        if (len(file.split(" ")) == 1):
            name = file.split(".")[0]
        else:
            name = file[0:file.index("(")].strip()


        change_to_30_fps(video_input_path=input_path, video_output_path=output_fps_path)
        extract_video(output_fps_path, name, sequence_id, input_path)

    os.chdir(initial_dir)

In [8]:
training_dir = f'{initial_dir}/data/videos_lensegua'
extract_coordinates_from_dir(training_dir)

!! No hand detected in  /Users/andreaamaya/Trabajo/megaproyecto_model/data/videos_lensegua/hola (10).mp4
entra en el except con  adjusted_fps_video.mp4


In [24]:
validation_dir = f'{initial_dir}/data/videos_val'
extract_coordinates_from_dir(validation_dir, is_val=True)

entra en el except con  querer.mp4
entra en el except con  medico.mp4
entra en el except con  si.mp4


KeyboardInterrupt: 

In [10]:
df = pd.DataFrame(frames)

In [11]:
df['sequence_id'] = df['sequence_id'].astype(int)

In [12]:
df.head()

,sequence_id,target,file,x_Left_hand_0,y_Left_hand_0,x_Left_hand_1,y_Left_hand_1,x_Left_hand_2,y_Left_hand_2,x_Left_hand_3,...,x_Right_hand_16,y_Right_hand_16,x_Right_hand_17,y_Right_hand_17,x_Right_hand_18,y_Right_hand_18,x_Right_hand_19,y_Right_hand_19,x_Right_hand_20,y_Right_hand_20
0,1,adios,/Users/andreaamaya/Trabajo/megaproyecto_model/...,0.827920,1.0,0.934036,0.978228,0.980444,0.942305,0.986786,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,adios,/Users/andreaamaya/Trabajo/megaproyecto_model/...,0.817225,1.0,0.919262,0.974164,0.969290,0.935011,0.982964,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,adios,/Users/andreaamaya/Trabajo/megaproyecto_model/...,0.774163,1.0,0.901714,0.977671,0.963947,0.940899,0.977280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,adios,/Users/andreaamaya/Trabajo/megaproyecto_model/...,0.800099,1.0,0.922746,0.969956,0.988565,0.930440,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,adios,/Users/andreaamaya/Trabajo/megaproyecto_model/...,0.796019,1.0,0.948179,0.958495,1.000000,0.915316,0.963509,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
def clean_target_name(target):
    if ".mp4" in target:
        return target.split(".")[0]
    else:
        return target

df['target'] = df['target'].apply(clean_target_name)

In [20]:
len(df.columns)

87

In [21]:
if (len(validation_sequence_ids) > 0):
    validation_path = f'{initial_dir}/data/validation.csv'
    validation_df = df[df['sequence_id'].isin(validation_sequence_ids)][["sequence_id", "target", "file"]]
    validation_df.to_csv(validation_path, index=False)
    
    train_path = f'{initial_dir}/data/train.csv'
    train_df = df[~df['sequence_id'].isin(validation_sequence_ids)][["sequence_id", "target", "file"]]
    train_df.to_csv(train_path, index=False)

In [22]:
data_path = f'{initial_dir}/data/data.csv'
df_final = df.drop("file", axis=1)
df_final.to_csv(data_path, index=False)